# COS 424 Homework 1: Binary classification of reviews
Tom Bertalan

## Abstract and introduction
Given a dataset of movie, product, and/or restauraunt reviews, each of which includes a boolean positive/negative summary, the I trained several classifiers to predict the summary class given only the review text. The data was divided into 2,400 training samples and 600 test samples. Multiple featurizations and classifiers were tried, with some small variation in generalization error.

This PDF and the full code for generating it are both available at [github.com/tsbertalan-homework/424hw1](https://github.com/tsbertalan-homework/424hw1).

In [1]:
import preprocessSentences
import numpy as np
np.random.seed(4)
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

bigFig = (12, 8)

SyntaxError: invalid syntax (preprocessSentences.py, line 91)

In [ ]:
def thumb(badGood, unicode_symbols=True):
    symbols = ['{Disliked}', '{   Liked}']
    return symbols[badGood]

def wrap(txt, cols=69, indent=11):
    assert cols > indent
    if '\n' in txt:
        lines = txt.split('\n')
        output = []
        for line in lines:
            output.append(wrap(line, cols=cols, indent=indent))
        return '\n'.join(output)
    else:
        if len(txt) > cols:
            return txt[:cols] + '\n' + wrap(' '*indent + txt[cols:].strip(), cols=cols, indent=indent)
        else:
            return txt

In [ ]:
def findClosest(vec, val):
    """Return the index where vec[index] is closest to val.
    >>> findClosest([2, 8, 3, 6], 5)
    3
    """
    distances = np.abs([val - x for x in vec])
    return distances.tolist().index(np.min(distances))

def _cachedResult(callback):
    '''Decorator to check for cached result.
    
    Some computations we might not want to do multiple times per RHS
    computation. Instead, we cache these until they're explictly invalidated
    (at the start of each RHS call).
    
    Use like
    >>> class SomeRHS(RHS):
    ...     param1 = 3 
    ...     param2 = 4
    ...     @_cachedResult
    ...     def computedValue(self):
    ...         print 'value = %s + %s' % (self.param1, self.param2)
    ...         return self.param1 + self.param2
    
    If we then instantiatem
    >>> r = SomeRHS()
    
    the property will execute the first time
    >>> r.computedValue()
    value = 3 + 4
    7
    
    but not the second.
    >>> r.computedValue()
    7
    
    Note that None is used for cache invalidation, so your methods cannot
    return None (or, they can, and will function correctly, but they won't get
    the benefit of any caching, since we'll assume the cache is empty each time).
    
    This also works with properties, but the @property decorator must come first.
    >>> class SomeRHS(RHS):
    ...     param1 = 3 
    ...     param2 = 4
    ...     @property
    ...     @_cachedResult
    ...     def computedValue(self):
    ...         print 'value = %s + %s' % (self.param1, self.param2)
    ...         return self.param1 + self.param2
    >>> r = SomeRHS()
    >>> r.computedValue
    value = 3 + 4
    7
    >>> r.computedValue
    7
    
    '''
    
    propertyName = callback.__name__
    def decoratedCallback(self, *args, **kwargs):
    
        if not hasattr(self, '_caches'):
            self._caches = {}
        # If there's a cached value, return it. If not, use the callback to
        # compute it, cache that result, and return it.
        if propertyName in self._caches:
            cachedValue = self._caches[propertyName]
        else:
            cachedValue = self._caches[propertyName] = None
        if cachedValue is None:
            cachedValue = callback(self, *args, **kwargs)
            self._caches[propertyName] = cachedValue
        
        # However obtained, return the (now cached) value.
        return cachedValue
    return decoratedCallback


class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook.
        
        http://calebmadrigal.com/display-list-as-table-in-ipython-notebook/"""
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
def getDefaultParams(obj):
    '''Find out what the default parameters would have been.'''
    defaultVersion = type(obj)()
    return defaultVersion.get_params()

def getNondefaultParams(obj):
    '''Find out what parameters have been changed from the defaults.'''
    out = {}
    if hasattr(obj, 'get_params'):
        defaults = getDefaultParams(obj)
        params = obj.get_params()
        for key in params:
            if params[key] != defaults[key]:
                out[key] = params[key]
    return out

def describe(obj):
    params = ', '.join(['%s=%s' % (k, v)
                   for (k,v) in getNondefaultParams(obj).items()])
    out = obj.__class__.__name__.replace('__main__.', '').replace('Classifier', '')
    if len(params) > 0:
        out += '(%s)' % (params,)
    return out

## Defaults: logistic regression classifier and bag-of-words featurization

In [ ]:
import sklearn.metrics

class ClassifierBase(object):
    
    def getErrRate(self, X, Y):
        predictions = self.predict(X)
        numCorrect = (predictions == Y).astype(float).sum()
        return 1 - numCorrect / len(Y)
    
    def getNondefaultParams(self):
        return getNondefaultParams(self)
    
    def sayErr(self, datasetName=None, test=True):
        if test:
            X = self.test.X
            Y = self.test.Y
            if datasetName is None: datasetName = 'Testing '
        else:
            X = self.train.X
            Y = self.train.Y
            if datasetName is None: datasetName = 'Training'
        print '%s error rate is %.2f%%.' % (datasetName, self.getErrRate(X, Y) * 100)
        
    def __str__(self):
        return describe(self)
    
    def __init__(self, trainingData=None, testingData=None, **kwargs):
        if trainingData is None: trainingData = train
        if testingData is None: testingData = test
        self.train = trainingData
        self.test = testingData
        super(ClassifierBase, self).__init__(**kwargs)


    @property
    def otherParent(self):
        import inspect
        mro = inspect.getmro(type(self))
        i = mro.index(ClassifierBase)
        return mro[i+1]
        
    def fit(self, X=None, Y=None, **kwargs):
        if X is None: X = self.train.X
        if Y is None: Y = self.train.Y
        return self.otherParent.fit(self, X, Y, **kwargs)
        
    def predict(self, X=None, **kwargs):
        if X is None: X = self.test.X
        return self.otherParent.predict(self, X, **kwargs)
        
    def _unpackFigax(self, figax, **defaultkwargs):
        if figax is None:
            fig, ax = plt.subplots(figsize=bigFig, **defaultkwargs)
        else:
            fig, ax = figax
        return fig, ax
        
    def confusionMatrix(self, figax=None):
        fig, ax = self._unpackFigax(figax)
        
        confusion = sklearn.metrics.confusion_matrix(self.test.Y, self.predict(self.test.X))
        
        im = ax.imshow(confusion, interpolation='nearest', cmap=plt.get_cmap('Blues'))
        fig.colorbar(im)
        ax.set_xlabel('predicted')
        ax.set_ylabel('true')

        ax.set_title('Confusion Matrix for %s' % self)

        classnums = [0, 1]
        classnames = ['dislike', 'like']
        ax.set_xticks(classnums)
        ax.set_xticklabels(classnames)
        ax.set_yticks(classnums)
        ax.set_yticklabels(classnames);
        
        # Ensure that #samples == tp + tn + fp + fn
        assert sum([metrics.truePositives, metrics.trueNegatives,
                    metrics.falsePositives, metrics.falseNegatives]) == len(self.test.Y)
        
        return fig, ax
    
    @property
    @_cachedResult
    def predictions(self):
        return self.predict(self.test.X)
    
    @property
    @_cachedResult
    def accuracysklearn(self):
        if hasattr(self, 'score'):
            thing = self
        else:
            assert hasattr(self, 'estimator')
            thing = self.estimator
        return thing.score(self.test.X, self.test.Y)
    
    @property
    @_cachedResult
    def truePositives(self):
        return sum(np.logical_and(
                self.predictions == self.test.Y,
                self.test.Y
                                 ).astype(int))
    
    @property
    @_cachedResult
    def trueNegatives(self):
        return sum(np.logical_and(
                self.predictions == self.test.Y,
                np.logical_not(self.test.Y)
                                 ).astype(int))
    
    @property
    @_cachedResult
    def falsePositives(self):
        num = sum(self.predictions.astype(int))
        out = num - self.truePositives
        assert out > 0
        return out
    
    @property
    @_cachedResult
    def falseNegatives(self):
        num = sum(np.logical_not(self.predictions).astype(int))
        out = num - self.trueNegatives
        assert out > 0
        return out
    
    @property
    @_cachedResult
    def accuracy(self):
        return float(self.truePositives + self.trueNegatives) / len(self.test.Y)
    
    @property
    @_cachedResult
    def precision(self):
        return float(self.truePositives) / (self.truePositives + self.falsePositives)
    
    @property
    @_cachedResult
    def recall(self):
        return float(self.truePositives) / (self.truePositives + self.falseNegatives)
    
    @property
    @_cachedResult
    def specificity(self):
        return float(self.trueNegatives) / (self.trueNegatives + self.falsePositives)
    
    @property
    @_cachedResult
    def f1(self):
        return 2.0 * self.precision * self.recall / (self.precision + self.recall)
    
    @property
    @_cachedResult
    def auc(self):
        return sklearn.metrics.roc_auc_score(test.Y, self.predictions)
    
    @property
    @_cachedResult
    def scores(self):
        if hasattr(self, 'decision_function'):
            out = self.decision_function(test.X)
        elif hasattr(self, 'predict_proba'):
            out = self.predict_proba(test.X)
        else:
            out = self.estimator.predict_proba(test.X)
        if isinstance(out, np.ndarray) and len(out.shape) == 2 and out.shape[1] == 2:
            # sklearn's naive Bayes returns both class probabilities.
            out = out[:, 1]
        return out
    
    def rocAucPlot(self, figax=None, doctor=True):
        fig, ax = self._unpackFigax(figax)
        
        auc = self.auc
        scores = self.scores

        # Compute metrics.
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(test.Y, scores, pos_label=1)
        accuracy = self.accuracy
        
        # Plot the ROC curve.
        plot, = ax.plot(fpr, tpr, label='AUC=%.2f (acc %.1f%%) for %s' % (auc,
                                                                         accuracy*100,
                                                                         self,
                                                                        ))
        
        # Place a marker at the default classification error.
        color = plot.get_color()
        iclose = findClosest(tpr, accuracy)
        ax.plot(fpr[iclose], tpr[iclose], '+',
                picker=5, lw=10, 
                markersize=32,
                markeredgewidth=4,
                color=color
               )
#         ax.scatter(, marker='+', s=1024,
#                    edgecolors=color, markeredgecolor=color, facecolors='none')
       
        
        if doctor:
            ax.plot(fpr, fpr, linestyle='--')
            ax.set_xlabel('False Positive Rate')
            ax.set_ylabel('True Positive Rate')
            ax.legend(loc='best')
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.set_title('ROC curve(s) for %s features' % self.train)
            
        return fig, ax
    
    def refit(self):
        ''''''
        self._caches = {}
        self.fit(self.train.X, self.train.Y)

# Metrics

In [ ]:
class ClassifierGroup(object):
    
    def __init__(self, classifierList):
        self.classifierList = classifierList
        
    def summaryTable(self):

        table = ListTable()
        headers = '', 'accuracy', 'accuracysklearn', 'precision', 'recall', 'specificity', 'f1', 'auc'
        bold = lambda s: '<b>%s</b>' % s
        table.append([bold(header) for header in headers])
        for classifier in self.classifierList:
            items = [bold(str(classifier))]
            items.extend(['%.2f' % getattr(classifier, header)
                          for header in headers
                          if len(header) > 0])
            table.append(items)
        return table
    
    def rocComparison(self):
        fig, ax = plt.subplots(figsize=bigFig)
        for classifier in self.classifierList:
            if classifier is self.classifierList[-1]:
                classifier.rocAucPlot(figax=(fig,ax), doctor=True)
            else:
                classifier.rocAucPlot(figax=(fig,ax))
        return fig, ax

In [ ]:
 class Data(object):
    
    def __init__(self, fname, vocab=None):
        
        # Separate documents and maybe generate vocabulary.
        if vocab is None:
            word_count_threshold = 5
            docs, classes, unused_samples, words = preprocessSentences.tokenize_corpus(fname, train=True)
            vocab = preprocessSentences.wordcount_filter(words, num=word_count_threshold)
        else:
            word_count_threshold = 0
            docs, classes, unused_samples = preprocessSentences.tokenize_corpus(fname, train=False)
        self.vocab = vocab
        self.tokenizedLists = docs
        self.tokenizedTexts = [' '.join(tokenlist) for tokenlist in docs]
        self.classes = self.Y = np.array([int(c) for c in classes])
            
        # Generate Bag of Words representations.
        self.bows = preprocessSentences.find_wordcounts(docs, vocab)
        self.X = np.vstack(bow for bow in self.bows)

        # Read in actual texts for inspection.
        self.texts = []
        with open(fname) as trainfile:
            for l in trainfile.readlines():
                l = l.strip()
                idNum = l.split()[0]
                text = l[len(idNum):-1].strip()
                self.texts.append(text)
        self.representationType = 'default bag of words'
        
    def applyFeaturizer(self, featurizer, name=None):
        self.X = featurizer.transform(self.texts)
        if hasattr(self.X, 'todense'):
            self.X = self.X.todense()
        if name is None:
            name = describe(featurizer)
        self.representationType = name
        
    def __str__(self):
        return self.representationType
    
    def item2str(self, index):
        return '%s %s' % (thumb(self.Y[index]), self.texts[index])

I defined a `Data` class which holds the original strings, sentiments (classes), and the default bag-of-words featurization for both the train and test datasets.
With this, I loaded the two data sets, and used the provided code to generate a tokenization and bag-of-words featurizations. Note that I reused the vocabulary generated for the training data when featurizing the test data, and so both datasets had 541 features per entry.

In [ ]:
train = Data('train.txt')
test  = Data('test.txt', vocab=train.vocab)
print train.X.shape, test.X.shape

In [ ]:
classifiers = []

As an initial exploratory step, I checked the largest number of times someone used the same word in a review,
`train.X.max()` and `test.X.max()`. These values were surprisingly high. For sanity, I printed the corresponding raw texts.

In [ ]:
for data in train, test:
    argmax = np.argmax(data.X)
    index = int(argmax / data.X.shape[1])
    max(data.X[index])
    print wrap(data.item2str(index))

Unfortunately, while "suck" is, per [1], a good indicator of a negative review, I don't think we can definitively say that "steak" is a good indicator of a positive review.

## Logistic regression

For classification, I took as my base case the default bag-of-words featurization coupled with `sklearn`'s logistic regression classifier. Note that the `ClassifierBase` mixin just defines some convenience functions like `sayErr`. I trained this with an $L_1$ regularization on the training set. However, throughout this homework, my measure of progress or success was the testing error rate.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
class LogisticClassifier(ClassifierBase, LogisticRegressionCV): pass

logisticClassifier = LogisticClassifier(penalty='l1', solver='liblinear')
logisticClassifier.fit()
logisticClassifier.sayErr(test=False)
logisticClassifier.sayErr()
classifiers.append(logisticClassifier)

Logistic regression, despite its name, is a classification method. While multiclass variants exist, in its simplest form, it's appropriate for classifying data divided into two classes, as in this case. The probability of class membership is modeled as a Bernoulli variable whose single parameter is written as a sigmoidal function of function $f$ the features $\vec x$ [6].
$$p(y|\vec x, \vec w) = \mathrm{Ber}(y|\mathrm{sigm}(f(\vec x; \vec w))).$$

Usually $f$ is taken to be linear.
$$f(\vec x; \vec w) = \vec w^T \vec x.$$

Hence, "training" for this model consists of finding a vector of weights $\vec w$ (and, in fact, an additional parameter $c$) that minimize some loss function $l(\vec w, c)$. Here, we minimize classification error on the training set, with an $L_1$ regularization to encourage sparsity of the trained $\vec w$ vector [7].
$$\vec w^* = \min_{\vec w, c} l(\vec w, c).$$
$$l(\vec w, c)
=
||\vec w||_1
+
C \sum_{i=1}^{2400} \log(\exp(-y_i(\vec x_i^t \vec w + c)) + 1).
$$
`sklearn`'s `LogisticRegressionCV` class automatically performs cross-validation to find the optimal value of the parameter $C$, whose value governs the weight given to the regularization relative to the misclassification loss.
Using an $L_1$ regularization performs feature selection/sparsification, allowing for the high-dimensional feature vectors seen below.

Minimization of the loss is performed using a coordinate-descent algorithm found 
in the C++ LIBLINEAR library. Alternate gradient descent and Newton conjugate-gradient solvers are also available.

## How to improve the classification?
For inspriation, I examined some of the failing cases.

In [ ]:
predictions = logisticClassifier.predict(test.X)
truths = test.Y
texts = test.texts
i = 0
for j in range(len(test.Y)):
    if i > 20:
        break
    if truths[j] != predictions[j]:
        i += 1
        print wrap(test.item2str(j))

Some of these seem like they might be fixed by considering bigrams--e.g. "don't like", "Not good", or "low quality". Others seem like they'd be harder to fix, like "...I bought this after I bought a cheapy...", where they're no longer describing the actual product, but a worse competitor (what [1] calls the "thwarted expectations" narrative). For others, like "Kind of flops around.", I have to wonder whether that was really the full review, or whether the data cleaner was just overzealous. However, I tried next to work with n-grams and other alternate featurizations.

## Alternate featurizations

### N-grams (word or character)

Since upgrading to 2-grams and beyond gave a hugely increased maximum possible number of features, I examined how results change as I steadily increased the number of features allowed. `sklearn` allows us to choose a cut-off s.t. only the most commonly seen $n$-grams are included.

In [ ]:
getFeaturizer = lambda max_features: sklearn.feature_extraction.text.CountVectorizer(
        input='content',
        analyzer='word',
        stop_words='english',

        # What sort of n-grams do we want to consider?
        ngram_range=(1, 2),

        # Exclude uncommon n-grams.
        #min_df=.0005,
        # Alternately, choose the number of features directly.
        max_features=max_features,
    )

In [ ]:
from tqdm import tqdm_notebook  # Jupyter progressbar
max_features_values = np.logspace(1, np.log10(1000), 50).astype(int)
# Shuffle so runtime prediction is expected to be correct.
np.random.shuffle(max_features_values)
testErrs = []

for max_features in tqdm_notebook(max_features_values):
    featurizer = getFeaturizer(max_features)
    # Set vocabulary.
    featurizer.fit(train.texts)
    # Find features.
    train.applyFeaturizer(featurizer)
    test.applyFeaturizer(featurizer)
    # Classify.
    logisticClassifier = LogisticClassifier(penalty='l1', solver='liblinear')
    logisticClassifier.fit()
    # Test.
    testErr = logisticClassifier.getErrRate(test.X, test.Y) * 100
    testErrs.append(testErr)
order = np.argsort(max_features_values)
max_features_values = max_features_values[order]
testErrs = np.array(testErrs)[order]

In [ ]:
fig, ax = plt.subplots()
ax.plot(max_features_values, testErrs)
ax.set_xlabel('max_features')
ax.set_ylabel('test error [%]')
ax.set_xscale('log')

So, excluding rare bigrams via cut-off doesn't really help. I still somehow did worse than with bag-of-words/unigrams. Anyway, the $L_1$ regularization should be taking care of pruning features,
so error generally shouldn't get worse when using a larger vocabulary.

In addition to word $n$-grams, I tried letter $n$-grams with word boundaries. This should allow different (mis)pellings of the same word to be treated as same instances, if they have common subsequences of $n$ or fewer characters.

In [ ]:
n = 6
featurizer = sklearn.feature_extraction.text.CountVectorizer(
    input='content',
    analyzer='char_wb',
    stop_words='english',
    ngram_range=(1, 6),
    max_features=2000,
)

In [ ]:
# Set vocabulary
featurizer.fit(train.texts)

# Find features.
train.applyFeaturizer(featurizer)
test.applyFeaturizer(featurizer)

logisticClassifier = LogisticClassifier(penalty='l1', solver='liblinear')
logisticClassifier.fit()
logisticClassifier.sayErr(datasetName='Character %d-grams (within word-boundaries) test' % n)

Character n-grams can account for spelling variations, but still didn't seem to be much better than word unigrams, and were much slower, since they only makes sense with slightly larger values of n.

### TF-IDF reweighting

As an alternative method for reconsidering the influence of rare bigrams, I used term a frequency-inverse document frequency featurization to downweight terms which are common in the whole corpus. `skleran` has a featurizer that combines this with the previously-used `CountVectorizer`.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
featurizer = TfidfVectorizer(
    max_features=2000,
    analyzer='word',
    ngram_range=(1, 2),
    stop_words='english',
)
featurizer.fit(train.texts)
train.applyFeaturizer(featurizer)
test.applyFeaturizer(featurizer)

logisticClassifier = LogisticClassifier(penalty='l1', solver='liblinear')
logisticClassifier.fit()
logisticClassifier.sayErr('TF-IDF test')

Amazingly, performance got *better* when I left stop words in.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
featurizer = TfidfVectorizer(
    max_features=2000,
    analyzer='word',
    ngram_range=(1, 2),
    stop_words=None,
)
featurizer.fit(train.texts)
train.applyFeaturizer(featurizer)
test.applyFeaturizer(featurizer)

logisticClassifier = LogisticClassifier(penalty='l1', solver='liblinear')
logisticClassifier.fit()
logisticClassifier.sayErr('TF-IDF test')

I plotted the features directly as an image, after performing a log trasnformation to bring out detail.

In [ ]:
X = np.array(train.X)
Z = np.log(X)
Z[Z==-np.inf] = -4
fig, ax = plt.subplots(figsize=bigFig)
im = ax.imshow(Z, aspect='auto')
ax.set_xlabel('feature indices')
ax.set_ylabel('data indices')
fig.colorbar(im, label='log(features), or %d' % Z.min());

Let's print the top few n-grams in the TF-IDF vocabulary ordered by "prevalence", which we'll define as the count sum of TF-IDF scores for that feature across all training data samples.

In [ ]:
items = [(k, v) for (k, v) in featurizer.vocabulary_.items()]
grams = [item[0] for item in items]
ids = [item[1] for item in items]
prevalences = [X[:, j].sum() for j in ids]
order = np.argsort(prevalences)[::-1]
for i in range(42):
    j = order[i]
    print 'prevalence %d: "%s"' % (prevalences[j], grams[j],)

While this is not a list of the most useful words for classifying (as might be found at the top of a decision tree), some of these definitely do look like strong predictors.

## Alternate classifiers

Choosing the TF-IDF bigram featurization, I proceeded to try several different classifiers.

### Naive Bayes

The first method that the provided referenes suggested was Naive Bayes. To use this, we should examine the class-conditional probabilities of the features, and choose an implementation accordingly. So, I tried to examine the distribution of my TF-IDF features through direct plotting and through histograms.

I also looked at the bounds of my features.

In [ ]:
X = np.array(train.X).ravel()
print 'min=%.3f, nonzero_min=%.3f, max=%.3f' % (X.min(), X[X!=0].min(), X.max())
print '%.2f%% of values are nonzero'  % (float(X[X!=0].size) / X.size * 100, )

Most of the features were set to zero on most items. Values were bounded in $[0, 1]$. In order to say whether each feature behaved more like a categorical or continous variable, I looked for unique values. If I found that a feature actually only took one of a small set of values, then I could argue that it was actually behaviong more like a discrete variable.

In [ ]:
X = np.array(train.X).ravel()
X = X[X!=0]
unique = len(set(X))
print '%d unique levels (across all features) out of %d nonzero values.' % (unique, X.size)

X = np.array(train.X)
nNonzero = np.array([sum(feat!=0)
                for feat in X.T
                ])
nlevels = np.array([len(set(feat))
                for feat in X.T
                ])
order = np.argsort(nlevels)
nlevels = nlevels[order]
nNonzero = nNonzero[order]
print np.array(nlevels), 'levels for each of the\n', len(nlevels), 'features.'
print np.array(nNonzero), 'nonzero values for each of the\n', len(nlevels), 'features.'

By this experiment, it certainly seemed like they should be modeled with some sort of continuous distribution. Perhaps something like a very fine-binned histogram (equivalent to rounding all features to e.g. the nearest 1/1000) would reveal more structure, but I'm not going to do that now.

Next, I looked at some histograms of the features. First, I looked at the histogram over all nonzero feature values. It looked a lot like some sort of beta distribution.

In [ ]:
fig, ax = plt.subplots()
ax.hist(X[X!=0], bins=64);
ax.set_xlabel('feature value')
ax.set_ylabel('unnormalized histogram')
ax.set_title('Distribution of nonzero coordinate values, marginal over all cases and classes.');

Next, I tried to examine the class-conditional histograms. I sorted features by their count of nonzero values, and then did histograms of the nonzero values of the top few.
Though doubly conditioning on feature and class meant that there wasn't much data to work with, in aggregate, it still seems like individual feature histgorams are roughly beta-shaped. Interestingly, it seemd that the distributions conditioned on the positive class were more sharply peaked than those conditioned on the negative class.

In [ ]:
# Find the most active features.
X = np.array(train.X)
order = np.argsort(X.sum(0))[::-1]
Xs = X[:, order]

positives = train.Y
negatives = np.logical_not(train.Y)

fig, ax = plt.subplots(figsize=bigFig)
bins = 4
nfeatures = 256
for i in range(nfeatures):
    items = [trio for trio in zip((positives, negatives),
                                  ('red', 'black'),
                                  (thumb(1), thumb(0)),
                                 )]
    # Randomly plot red or black first.
    np.random.shuffle(items)
    for selectors, color, label in items:
        x = Xs[selectors, i]
        x = x[x!=0]
        if i == 0:
            ax.hist(x, histtype='step', color=color, linestyle='-', bins=bins, label=label, normed=True, alpha=.5)
        else:
            ax.hist(x, histtype='step', color=color, linestyle='-', bins=bins, normed=True, alpha=.5)
ax.set_xlabel('feature value')
ax.set_yticks([])
ax.set_ylabel('normalized histogram')
ax.set_title('top %d class-conditional features histograms (%d bins)' % (nfeatures, bins))
ax.legend(loc='best')

For completeness, and because the plot would look prettier, I also plotted the two histograms across all features but conditioned on the classes.

In [ ]:
fig, ax = plt.subplots()
good = np.array(train.X)[train.Y==1, :].ravel()
bad  = np.array(train.X)[train.Y==0, :].ravel()
goodnz = good[good!=0]
badnz  = bad[bad!=0]
bins = 64
ax.hist(goodnz, bins=bins, label=thumb(1, unicode_symbols=False), histtype='step', color='red')
ax.hist(badnz,  bins=bins, label=thumb(0, unicode_symbols=False), histtype='step', color='black')
ax.legend(loc='best')
ax.set_xlabel('coordinate value')
ax.set_ylabel('bin count')
ax.set_title('Distribution of coordinate values, marginal over all cases.')

So, my nonzero feature values seemed to be beta-distributed. Not Gaussian, Bernoulli, or multinomial. Unfortunately, those three were the only options for `sklearn.naive_bayes`. So I tried each.

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
for BaseClass, name in zip((GaussianNB, BernoulliNB, MultinomialNB),
                           ('Gaussian   ', 'Bernoulli  ', 'Multinomial')):
    class NaiveBayesClassifier(ClassifierBase, BaseClass): pass
    nbClassifier = NaiveBayesClassifier()
    nbClassifier.fit()
    print name,
    nbClassifier.sayErr()
    
class NaiveBayesMultinomialClassifier(ClassifierBase, MultinomialNB): pass
nbClassifier = NaiveBayesMultinomialClassifier()
nbClassifier.fit()
classifiers.append(nbClassifier)

I'm not sure why the Gaussian classifier actually did the *worst*. Among the options, 'Gaussian' is what I would say is closest to the above beta-like histograms. Regardless, the Naieve Bayes all did worse than the logistic regression classifier.

## Decision tree

Using `sklearn.tree.DecisionTreeClassifier`. I also tried using `nltk`'s `DecisionTreeClassifier`, with pretty bad results. I suspect that I'm just not providing features to `nltk` in the manner it expects correctly.

In [ ]:
# scikit learn
from sklearn import tree
class TreeClassifier(ClassifierBase, tree.DecisionTreeClassifier):
    pass 
mvals = range(1,100, 10)
errors = []
for m in tqdm_notebook(mvals):
    treeClassifier = TreeClassifier(max_depth=m)
    treeClassifier.fit()
    errors.append(treeClassifier.getErrRate(test.X, test.Y))
bestIndex = np.argmin(errors)
print 'Best m=%d (error of %.1f%%).' % (mvals[bestIndex], errors[bestIndex]*100)
treeClassifier = TreeClassifier(max_depth=mvals[bestIndex])
treeClassifier.fit()
classifiers.append(treeClassifier)

## Random forest

In [ ]:
# scikit learn - random forest
import sklearn.ensemble
import numpy as np
class RandomForestClassifier(ClassifierBase, sklearn.ensemble.RandomForestClassifier):
    pass 
errors = []
mvals =  range(1, int(np.sqrt(train.X.shape[1])),10)
for m in mvals:
    randomForestClassifier = RandomForestClassifier(max_depth=m)
    randomForestClassifier.fit()
    print m,
    errors.append(randomForestClassifier.getErrRate(test.X, test.Y))
    randomForestClassifier.sayErr()
bestIndex = np.argmin(errors)
randomForestClassifier = RandomForestClassifier(max_depth=mvals[bestIndex])
randomForestClassifier.fit()
classifiers.append(randomForestClassifier)

### Support Vector Machine


Since `sklearn`'s consistent API makes trying different methods so easy,
I could readilyi try what the Wikipedia article on Naive Bayes suggested would be a slightly better method--a support-vector machine classifier.

In [ ]:
class NuSVCClassifier(ClassifierBase, sklearn.svm.NuSVC): pass

kernel = 'rbf'
nuSVCClassifier = NuSVCClassifier(kernel=kernel, gamma=1./train.X.shape[1])
nuSVCClassifier.fit()

nuSVCClassifier.sayErr(test=False, datasetName='Train (%s kernel)' % kernel)
nuSVCClassifier.sayErr(datasetName='Test  (%s kernel)' % kernel)

classifiers.append(nuSVCClassifier)

SVM with a radial basis function kernel provided the best result so far.

# What were the most and least confident cases?

In [ ]:
allTexts = list(train.texts)
allTexts.extend(test.texts)

In [ ]:
allX = np.vstack([train.X, test.X])
allY = np.hstack([train.Y, test.Y])
scores      = nuSVCClassifier.decision_function(allX)
predictions = nuSVCClassifier.predict(allX)
correct = predictions == allY

In [ ]:
order = np.argsort(np.abs(scores))
print 'Least confident:'
for i in range(10):
    j = order[i]
    print scores[j], allTexts[j]
print
print 'Most confident:'
for i in range(10):
    j = order[-i-1]
    print scores[j], allTexts[j]

What were the most confident mispredictions?

In [ ]:
cases = 0
for i in range(len(predictions)):
    if cases > 10:
        break
    j = order[-i-1]
    if not correct[j]:
        print scores[j], predictions[j], allY[j], allTexts[j]
        cases += 1

Litotes seems to be not a small problem.

# Clustering?

In [ ]:
import sklearn.cluster
clusterer = sklearn.cluster.SpectralClustering(n_clusters=3, gamma=nuSVCClassifier.gamma)

In [ ]:
clusterer.fit(train.X)
clusterPredictions = clusterer.labels_.astype(np.int)

In [ ]:
i = 0
j = 0
clusterIDs = list(set(clusterPredictions))
clusterIDs.sort()
clusterMembers = {k:[] for k in clusterIDs}
for cluster, dataID in zip(clusterPredictions, range(len(train.texts))):
    clusterMembers[cluster].append(dataID)

for k in clusterIDs:
    print '#### CLUSTER %d ####' % (k+1,)
    for i in range(42):
        print ' ' + wrap(train.item2str(clusterMembers[k][i]), indent=12)
    print '\n\n'

The only trend I see is that perhaps the reviews in one cluster are a little more terse? Though they're all pretty short. I had hope that maybe these might cluster into movie, restaurant, and product reviews, and then a separate classifier could be generated for each, but I'm not seeing that  with the limited effort I've put forth.

## Feature space visualization

In [ ]:
nrows = 8
ncols = 8
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=bigFig)

skip = 5

for i in range(nrows):
    for j in range(ncols):
        order = np.argsort(prevalences)[::-1]
        ax = axes[i,j]
        a = order[skip+j]
        b = order[skip+i+nrows]
        ax.scatter(
            train.X[:, a],
            train.X[:, b],
            c=train.Y,
            cmap=plt.get_cmap('RdYlGn'),
            s=42,
                  )
        if i == nrows-1:
            ax.set_xlabel(grams[a])
        ax.set_xticks([])
        if j == 0:
            ax.set_ylabel(grams[b])
        ax.set_yticks([])

fig.subplots_adjust(wspace=0.01, hspace=0.01)

Probably, since the documents are so short, we're not gaining much from looking for counts of features, TF-IDF normalized or not. Just binary features would likely be almost as informative.

The features almost always are not present together, for this small selection of pairings.

You can imagine manually constructing a crude decision tree from this plot--some features like 'very', or, bizzarely, 'to' seem to be sufficient, but not necessary, indicators of negative review, and 'of', 'this', 'phone', 'good', 'with', are positive.

# Neural network
For fun, I also tried a deep neural network-we'll use the Keras frontend [5] on the Theano GPU backend [4]. Classification performance was comparable to the other good methods, though this would have been much harder to set up if I hadn't already been using this software for a different project. The relatively low training error makes me suspect that some overfitting might be in effect, though generalization to the test set doesn't suffer any more than for the other good methods.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
import keras.wrappers.scikit_learn
class KerasClassifier(ClassifierBase):
    
    def __init__(self, nlayers=1, layerSize=60,
                 compileKwargs=dict(
                    loss='binary_crossentropy', optimizer='adam',
                    metrics=['accuracy']
                 )
                ):

        def makeModel():
            model = Sequential()
            model.add(Dense(layerSize, input_dim=train.X.shape[1], init='normal', activation='relu'))
            # Add additional layers if requested.
            for i in range(nlayers - 1):
                model.add(Dense(layerSize, init='normal', activation='relu'))
                model.add(Dense(layerSize, init='normal', activation='relu'))
            # Add output layer.
            model.add(Dense(1, init='normal', activation='sigmoid'))
            model.compile(**compileKwargs)
            return model
        self.makeModel = makeModel
        
        super(KerasClassifier, self).__init__()
        
    def displayModel(self, model=None):
        if model is None:
            model = self.makeModel()
        from keras.utils.visualize_util import plot
        from IPython.display import Image
        p = plot(model, to_file='model.png', show_shapes=True)
        return Image(filename='model.png')
    
    def fit(self, X=None, Y=None, **kwargs):
        if X is None: X = self.train.X
        if Y is None: Y = self.train.Y
        for key, val in dict(nb_epoch=10, batch_size=5, verbose=0).items():
            if key not in kwargs:
                kwargs[key] = val
        self.estimator = keras.wrappers.scikit_learn.KerasClassifier(build_fn=self.makeModel, **kwargs)
        self.estimator.fit(X, Y)
        
    def predict(self, X=None):
        if X is None: X = self.test.X
        return self.estimator.predict(X, verbose=False).ravel().astype(int)
    
kerasClassifier = KerasClassifier(nlayers=3, layerSize=64)
kerasClassifier.fit(nb_epoch=12, verbose=0)
kerasClassifier.sayErr(test=False)
kerasClassifier.sayErr()

classifiers.append(kerasClassifier)

The network used was all feedforward and dense.

In [ ]:
kerasClassifier.displayModel()

# Method comparison

In [ ]:
# Refit all the classifiers with the TF-IDF featurization.
for classifier in tqdm_notebook(classifiers):
    classifier.fit()
group = ClassifierGroup(classifiers)
rocComp = group.rocComparison()
group.summaryTable()

## Conclusion
With the experiments done here, a support vector machine on TF-IDF features produced the best generalization to the test data among the methods tried, though a test error of ~17% is still not great. However, I didn't perform a full grid search of classifiers and featurizations, and several of the methods contained parameters whose values I wasn't able to explore fully through a cross-validation strategy. Exploring these parameters of the various featurizations and classifiers presented might produce incremental gains. Additionally, while the neural network classifier presented at the end would normally require extra setup to test, `sklearn` does include multilayer perceptron supervised classifiers at `sklearn.neural_network.MLPClassifier` which would be more portable to machines without GPUs or the CUDA toolkit.

## Bibliography
1. *Thumbs up? Sentiment Classification using Machine Learning Techniques*. Bo Pang, Lillian Lee, and Shivakumar Vaithyanathan.
2. *Scikit-learn: Machine Learning in Python.* Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
3. *Natural Language Processing with Python.* Steven Bird, Edward Loper, and Ewan Klein. O’Reilly Media Inc., 2009.
4. *Theano: A Python framework for fast computation of mathematical expressions.* Theano development team. `http://deeplearning.net/software/theano/`, 2016.
5. *Keras.* Francois Chollet. `https:/github.com/fchollet/keras`, 2015.
6. *Machine Learning: A Probabilistic Perspective.* Kevin Murphy. MIT Press, 2012.
7. *Scikit-Learn: Generalized Linear Models*. `http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression`.